In [1]:
import numpy as np                          # Basic Array and Numaric Operation
import os                                   # use to access the files 
import tarfile                              # use to extract dataset from zip files
import sys
import zipfile                              # to extract zip file


import torch                                # Provides basic tensor operation and nn operation
import torchvision                          # Provides facilities to access image dataset


import my_utils.loadDataset as dl           # create dataloader for selected dataset
import my_utils.loadModel as lm             # facilitate loading and manipulating models
import my_utils.trainModel as tm            # Facilitate training of the model
import my_utils.initialize_pruning as ip    # Initialize and provide basic parmeter require for pruning
import my_utils.facilitate_pruning as fp    # Compute Pruning Value and many things

### Prepare dataloader and set image properties. Check for cuda device and load model

In [2]:
# set the locationn of the dataset and trai and test data folder name
dl.setFolderLocation(datasets       ='/home/pragnesh/Dataset/',
                     selectedDataset='IntelIC/',
                     train          ='train',
                     test           ='test')
# set the imge properties
dl.setImageSize(224)
dl.setBatchSize = 2
dataLoader = dl.dataLoader()

#load the saved model if have any
load_path = "/home/pragnesh/Model/VGG_IntelIC_v2"
device1 = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
newModel = torch.load(load_path, map_location=torch.device('cpu'))

#if dont have any saved trained model download pretrained model for tranfer learning
# newmodel = lm.load_model(model_name='vgg16',number_of_class=6,pretrainval=False,
#                         freeze_feature=False,device_l=device1)

#print(newModel)

### Here we have 3 set of parameter.One for data loader, second for model and third are hyperparameter
### Train the model

In [ ]:
logFile = '/home/pragnesh/Dataset/Intel_Image_Classifacation_v2/Logs/ConvModelv2.log'
#tm.device = torch.device('cpu')
tm.fit_one_cycle(#set locations of the dataset, train and test data
                 dataloaders=dataLoader,trainDir=dl.trainDir,testDir=dl.testDir,
                 # Selecat a variant of VGGNet
                 ModelName='vgg16',model=newModel,device_l=device1,
                 # Set all the Hyper-Parameter for training
                 epochs=1, max_lr=0.01, weight_decay=0, L1=0, grad_clip=.1, logFile=logFile)

#Save the  trained model 
SavePath = '/home/pragnesh/Model/vgg16-v2'
torch.save(newModel, SavePath)

### Pruning Initialization

In [3]:
blockList   = ip.createBlockList(newModel)              #ip.getBlockList('vgg16')
featureList = ip.createFeatureList(newModel)
convIdx     = ip.findConvIndex(newModel)
module      = ip.getPruneModule(newModel)
prune_count = ip.getPruneCount(module=module,blocks=blockList,maxpr=.1)
print(f"Block List   = {blockList}\n"
      f"Feature List = {featureList}\n" 
      f"Conv Index   = {convIdx}\n"
      f"Prune Count  = {prune_count}"
      
     )
for i in range(len(module)):
    print(f"{i+1} :{module[i]}")

Block List   = [2, 2, 3, 3, 3]
Feature List = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M']
Conv Index   = [0, 2, 5, 7, 10, 12, 14, 17, 19, 21, 24, 26, 28]
Prune Count  = [1, 1, 3, 3, 9, 9, 9, 26, 26, 26, 51, 51, 51]
1 :Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
2 :Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
3 :Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
4 :Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
5 :Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
6 :Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
7 :Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
8 :Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
9 :Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
10 :Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
11 :Conv2d(512, 512, kernel_size=(3

### Pruning Process Starts From Here
##### compute score for pruning criteria for candidate convolution pruning layer

In [4]:
candidateConvLayer = []
st =0
en = 2
for i in range(st,en):
    candidateConvLayer.append(fp.compute_distance_score(module[i]._parameters['weight'],threshold=1))
print("\n\n\nHere is the :",len(candidateConvLayer))


Shape of the tensor: torch.Size([64, 3, 3, 3])
Print the Dims we want to keep: [0, 1]
norm shape = torch.Size([64, 3])
Number Of Features Map in current  layer l     = 64
Number Of Features Map in previous layer (l-1) = 3
...
Shape of the tensor: torch.Size([64, 64, 3, 3])
Print the Dims we want to keep: [0, 1]
norm shape = torch.Size([64, 64])
Number Of Features Map in current  layer l     = 64
Number Of Features Map in previous layer (l-1) = 64
................................................................


Here is the : 2


#### *Sort the list of score in the ascending order*

In [5]:
for i in range(len(candidateConvLayer)):
    fp.sort_kernel_by_distance(candidateConvLayer[i])

In [6]:
print(f'c={len(candidateConvLayer)}, c[] ={len(candidateConvLayer[0])},' 
      f'c[][] = {len(candidateConvLayer[0][0])},c[][][] = {len(candidateConvLayer[0][0][0])}')

c=2, c[] =3,c[][] = 190,c[][][] = 4


#### *Extract K Elements of ascending List of Score and they will be the minimum score*

In [7]:
prune_count = ip.getPruneCount(module=module,blocks=blockList,maxpr=0.1,)
newList = []
for i in range(len(candidateConvLayer)):
    newList.append(fp.get_k_element(channel_list=candidateConvLayer[i],k=prune_count[st+i])  )
print(f'c={len(newList)}, c[] ={len(newList[0])},' 
      f'c[][] = {len(newList[0][0])},c[][][] = {len(newList[0][0][0])}')

c=2, c[] =3,c[][] = 1,c[][][] = 4


### Now Execute the pruning

#### Define class to execute custom pruning

In [8]:
import torch.nn.utils.prune as prune
class KernalPruningMethod(prune.BasePruningMethod):
    PRUNING_TYPE = 'unstructured'
    def compute_mask(self, t, default_mask):
        mask = default_mask.clone()
        #mask.view(-1)[::2] = 0
        size = t.shape
        print(size)
        for k1 in range(len(newList[layer_number-st])):
            for k2 in range(len(newList[layer_number-st][k1])):
                i= newList[layer_number-st][k1][k2][1]
                j= newList[layer_number-st][k1][k2][0]
                
                #print(f"i= {i} , j= {j}")
                
                mask[i][j] = 0
        return mask
def kernal_unstructured(module, name):
    KernalPruningMethod.apply(module, name)
    return module

### *Pass the cadidate layer of Conv Net for custom pruning*

In [9]:
layer_number = 0
for i in range(st,en):
    layer_number = i
    kernal_unstructured(module[i], name = 'weight')
    layer_number += 1

torch.Size([64, 3, 3, 3])
torch.Size([64, 64, 3, 3])


##### commit the pruning parameter 

In [10]:
for i in range(st,en):
    prune.remove(module[i], 'weight')

### Deep Copy Code

In [11]:
def updateFeatureList(featureList,prune_count,start=0,end=):
    j=0
    i=st
    while j < en:
        if featureList[i] == 'M':
            i+=1
            continue
        else:
            featureList[i] = featureList[i] - prune_count[j]
            j+=1
            i+=1
    return featureList
featureList= updateFeatureList(featureList=featureList, prune_count=prune_count,start=st,end=en)


In [12]:
print(featureList)

[63, 63, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M']


In [13]:
#featureList = ip.createFeatureList(newModel)

In [14]:
tempModel = lm.create_vgg_from_feature_list(featureList)
# print(tempModel)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 63, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(63, 63, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(63, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [15]:
lm.freeze(model=tempModel,model_name='vgg16')

In [16]:
def deepCopy(destModel,sourceModel,featureList):
    count=0
    for i in range(len(sourceModel.features)):
        if str(sourceModel.features[i]).find('Conv') != -1:
            size = sourceModel.features[i]._parameters['weight'].shape
            for fin in range(size[1]):
                j=0
                for fout in range(size[0]):
                    if torch.norm(sourceModel.features[i]._parameters['weight'][fout][fin]) != 0:
                        #print(f'i ={i} , j={j}, fin = {fin}, fout= {fout}')
                        if j>=featureList[count]:
                            break
                        destModel.features[i]._parameters['weight'][j][fin]=sourceModel.features[i]._parameters['weight'][fout][fin]
                        j = j+1
            count = count +1
deepCopy(destModel=tempModel,sourceModel=newModel,featureList=featureList)

i =0 , j=0, fin = 0, fout= 0
i =0 , j=1, fin = 0, fout= 1
i =0 , j=2, fin = 0, fout= 2
i =0 , j=3, fin = 0, fout= 3
i =0 , j=4, fin = 0, fout= 4
i =0 , j=5, fin = 0, fout= 5
i =0 , j=6, fin = 0, fout= 6
i =0 , j=7, fin = 0, fout= 7
i =0 , j=8, fin = 0, fout= 8
i =0 , j=9, fin = 0, fout= 9
i =0 , j=10, fin = 0, fout= 10
i =0 , j=11, fin = 0, fout= 11
i =0 , j=12, fin = 0, fout= 12
i =0 , j=13, fin = 0, fout= 13
i =0 , j=14, fin = 0, fout= 14
i =0 , j=15, fin = 0, fout= 15
i =0 , j=16, fin = 0, fout= 16
i =0 , j=17, fin = 0, fout= 17
i =0 , j=18, fin = 0, fout= 18
i =0 , j=19, fin = 0, fout= 20
i =0 , j=20, fin = 0, fout= 21
i =0 , j=21, fin = 0, fout= 22
i =0 , j=22, fin = 0, fout= 23
i =0 , j=23, fin = 0, fout= 24
i =0 , j=24, fin = 0, fout= 25
i =0 , j=25, fin = 0, fout= 26
i =0 , j=26, fin = 0, fout= 27
i =0 , j=27, fin = 0, fout= 28
i =0 , j=28, fin = 0, fout= 29
i =0 , j=29, fin = 0, fout= 30
i =0 , j=30, fin = 0, fout= 31
i =0 , j=31, fin = 0, fout= 32
i =0 , j=32, fin = 0, fout= 

i =2 , j=24, fin = 32, fout= 25
i =2 , j=25, fin = 32, fout= 26
i =2 , j=26, fin = 32, fout= 27
i =2 , j=27, fin = 32, fout= 28
i =2 , j=28, fin = 32, fout= 29
i =2 , j=29, fin = 32, fout= 30
i =2 , j=30, fin = 32, fout= 31
i =2 , j=31, fin = 32, fout= 32
i =2 , j=32, fin = 32, fout= 33
i =2 , j=33, fin = 32, fout= 34
i =2 , j=34, fin = 32, fout= 35
i =2 , j=35, fin = 32, fout= 36
i =2 , j=36, fin = 32, fout= 37
i =2 , j=37, fin = 32, fout= 38
i =2 , j=38, fin = 32, fout= 39
i =2 , j=39, fin = 32, fout= 40
i =2 , j=40, fin = 32, fout= 41
i =2 , j=41, fin = 32, fout= 42
i =2 , j=42, fin = 32, fout= 43
i =2 , j=43, fin = 32, fout= 44
i =2 , j=44, fin = 32, fout= 45
i =2 , j=45, fin = 32, fout= 46
i =2 , j=46, fin = 32, fout= 47
i =2 , j=47, fin = 32, fout= 48
i =2 , j=48, fin = 32, fout= 49
i =2 , j=49, fin = 32, fout= 50
i =2 , j=50, fin = 32, fout= 51
i =2 , j=51, fin = 32, fout= 52
i =2 , j=52, fin = 32, fout= 53
i =2 , j=53, fin = 32, fout= 54
i =2 , j=54, fin = 32, fout= 55
i =2 , j

IndexError: index 63 is out of bounds for dimension 0 with size 63

In [ ]:
size = tempModel.features[0]._parameters['weight'].shape
for i in range(size[0]):    
    for j in range(size[1]):
        print(tempModel.features[0]._parameters['weight'][i][j])

In [ ]:
newModel

In [ ]:
lm.freeze(tempModel,'vgg16')